## Example of running DeepRL



This notebook demonstrates how to use the `DeepRL` library to discover the governing partial differential equation (PDE) from data. We will use data generated from the Burgers' equation, initialize and train the model, and then visualize and analyze the results using the `deeprl_viz` toolkit.

In [1]:
import os
import sys
import numpy as np

current_dir = os.getcwd()
kd_main_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(kd_main_dir)
# Since the juppter's kernel might be running in a different directory
os.chdir(kd_main_dir) 

import scipy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='numpy.*')
warnings.filterwarnings("ignore", category=UserWarning, module='tensorflow.*')
from kd.model import DeepRL_Pinn
from kd.viz.discover_eq2latex import discover_program_to_latex 
from kd.viz.equation_renderer import render_latex_to_image
from kd.viz.deeprl_viz import *


## 1. Load and prepare data

We begin by loading the dataset for the Burgers' equation from a `.mat` file. The data consists of the solution $u(x, t)$ on a spatiotemporal grid. The `prepare_data` function handles loading, reshaping the data into coordinate pairs `(x, t)` and corresponding solution values `u`, and defining the domain bounds.

For this demonstration, the `DeepRL` model will use the full, regular grid data provided by the `import_inner_data` method for the 'Burgers' dataset.

In [2]:
def prepare_data():
    
    data = scipy.io.loadmat('./kd/data_file/burgers2.mat')
    t = np.real(data['t'].flatten()[:,None])
    x = np.real(data['x'].flatten()[:,None])
    Exact = np.real(data['usol']).T  # t first
    X, T = np.meshgrid(x,t)

    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.flatten()[:,None]              

    # Doman bounds
    lb = X_star.min(0)
    ub = X_star.max(0) 

    x_len = len(x)
    total_num = X_star.shape[0]
    sample_num = int(total_num*0.1)
    print(f"random sample number: {sample_num} ")
    ID = np.random.choice(total_num, sample_num, replace = False)
    X_u_meas = X_star[ID,:]
    u_meas = u_star[ID,:]  
    return X_u_meas,u_meas, lb,ub

x,y,lb,ub = prepare_data()

model = DeepRL_Pinn(
    n_samples_per_batch = 1000, # Number of generated traversals by agent per batch
    binary_operators = ["add_t", "mul_t", "div_t", "diff_t","diff2_t",],
    unary_operators = ['n2_t'],
)
np.random.seed(42)

random sample number: 2585 


# 2. Train the model

In [ ]:
step_output = model.fit(x, y, [lb,ub])

print(f"Current best expression is {step_output['expression']} and its reward is {step_output['r']}")

-- BUILDING PRIOR START -------------
LengthConstraint: Sequences have minimum length 3.
                  Sequences have maximum length 64.
RepeatConstraint: [add_t] cannot occur more than 5 times.
TrigConstraint: [diff_t, diff2_t] cannot be a descendant of [diff_t, diff2_t].
SoftLengthPrior: No description available.
DiffConstraint_left: [x1] cannot be the left child of [diff_t, diff2_t].
DiffConstraint_right: [n2_t, add_t, mul_t, div_t, diff_t, diff2_t, u1] cannot be the right child of [diff_t, diff2_t].
DiffConstraint_des: [add_t] cannot be a descendant of [diff_t, diff2_t].
-- BUILDING PRIOR END ---------------



2025-06-14 09:08:05,288-root-INFO-ANN(
  (input_layer): Linear(in_features=2, out_features=20, bias=True)
  (mid_linear): ModuleList(
    (0-6): 7 x Linear(in_features=20, out_features=20, bias=True)
  )
  (out_layer): Linear(in_features=20, out_features=1, bias=True)
)
2025-06-14 09:08:05,288-root-INFO-Total params are 3021


Using device: cpu
NN evaluator training with data


2025-06-14 09:08:07,154-root-INFO-epoch: 500, loss_u: 0.00010991461749654263 , loss_val:0.00010436950833536685
2025-06-14 09:08:08,798-root-INFO-epoch: 1000, loss_u: 1.8108647054759786e-05 , loss_val:2.0109209799556993e-05
2025-06-14 09:08:10,663-root-INFO-epoch: 1500, loss_u: 1.013424298434984e-05 , loss_val:1.1639052900136448e-05
2025-06-14 09:08:12,434-root-INFO-epoch: 2000, loss_u: 4.209046437608777e-06 , loss_val:5.040526048105676e-06
2025-06-14 09:08:14,082-root-INFO-epoch: 2500, loss_u: 2.960152642117464e-06 , loss_val:3.2973950965242693e-06


# 3. Visualization

### 3.1 Discovered Equation and Expression Tree

The best program discovered by the model can be visualized both as a rendered LaTeX equation for clarity and as a symbolic expression tree. The tree shows the hierarchical structure of the equation, with operators as parent nodes and their arguments as children. This helps in understanding the composition of the discovered PDE.

In [ ]:
render_latex_to_image(discover_program_to_latex(step_output['program']))

In [ ]:
plot_expression_tree(model)

### 3.2 Training Diagnostics

These plots help diagnose the training process itself:

* **Density Plot:** This plot shows the distribution of rewards for all programs sampled at different stages of training (e.g., epochs 10, 30, 50). It helps visualize how the population of candidate solutions converges towards the high-reward region.
* **Evolution Plot:** This plot shows the evolution of rewards for the programs sampled in each training epoch. We can see how the agent improves over time, consistently finding programs with higher rewards as training progresses.


In [ ]:
plot_density(model)
# plot_density(model, epoches = [10,30,50])

In [ ]:
plot_evolution(model)

### 3.3 Residual Analysis

Residual analysis is a critical step to diagnose model fit. The **physical residual** is defined as $Residual = u_t - RHS$, where $u_t$ is the true time derivative and $RHS$ is the right-hand side of our discovered equation.

* **Spatiotemporal Distribution (Left):** This plot shows *where* the errors occur across the space-time domain. Instead of being randomly scattered, we observe a distinct, non-random pattern. A blue region (negative residuals) propagates through the domain, indicating where our discovered equation systematically **over-predicts** the true time derivative ($u_t$). This is flanked by red regions where the model **under-predicts**. The vast white area signifies that the error is extremely close to zero for most points.

* **Residual Distribution (Right):** This histogram shows the overall distribution of errors. The distribution is dominated by an **extremely sharp and high peak at zero**, indicating that the model's prediction is highly accurate for the vast majority of spatiotemporal points. The slight spread around the peak corresponds to the small, structured errors visible in the left plot.

In [ ]:
plot_pinn_residual_analysis(model, step_output['program'])

### 3.4 Field Comparison

This visualization provides a side-by-side comparison of the true field ($u_t$) and the field generated by the RHS of our discovered equation. By using a shared color scale, we can intuitively see where the model's predictions match the ground truth and where they differ. For many applications, this is a more direct and intuitive way to assess model performance than the residual plot.

In [ ]:
plot_pinn_field_comparison(model, step_output['program'])

### 3.5 Actual vs. Predicted

The 45-degree plot is a standard and powerful tool for evaluating regression models. Here, we plot the true values ($u_t$) on the x-axis against the predicted values (RHS) on the y-axis.

* **Overall Performance:** A perfect model would have all points lying exactly on the dashed `y=x` line. Our plot shows that for most points, the prediction is very accurate, as indicated by the tight clustering of points around the line. This confirms that the discovered equation captures the dominant dynamics of the system.
* **Systematic Error:** The most interesting feature is the loop-like structure in the center. This is not random noise but a **systematic error**. It suggests that the discovered equation, while very good, may be missing a term or a physical effect needed to fully capture the system's dynamics in certain regimes. This kind of structured error provides valuable insight for further model improvement.

In [ ]:
plot_pinn_actual_vs_predicted(model, step_output['program'])